In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
import pandas as pd
from pyscf import lib, gto, scf
import pyqmc
import h5py

This function computes the mean-field solution and saves the results to the file specified.

In [2]:
def mean_field(chkfile):
    mol = gto.M(atom = "He 0. 0. 0.", basis='bfd_vdz', ecp='bfd', unit='bohr')

    mf = scf.RHF(mol)
    mf.chkfile = chkfile
    mf.kernel()
    return mol, mf

Set up our wave function given a molecule and mean-field object

In [3]:
def qmcwf(mol, mf):
    jast, to_optj, freezej = pyqmc.default_jastrow(mol)
    slater = pyqmc.PySCFSlaterUHF(mol, mf)
    wf = pyqmc.MultiplyWF(slater,jast)
    freeze = {}
    for k in to_optj:
        freeze["wf2" + k] = freezej[k]
    to_opt = ["wf2" + x for x in to_optj]
    return wf, to_opt, freeze
    


This function runs VMC, recovering the mean-field objects from `chkfile`. These could be run in completely different processes.

If `hdffile` exists, then the configurations will be reloaded from it and the run will be continued. You can run this as many times as you like and the statistics will get better.

In [4]:
def runvmc(chkfile, wffile, hdffile):
    #recover the mean-field objects
    mol = lib.chkfile.load_mol(chkfile)
    mf = scf.RHF(mol)
    mf.__dict__.update(scf.chkfile.load(chkfile, 'scf'))

    #Set up our MC variables and wave function
    wf, _, _ = qmcwf(mol,mf)
    hdf = h5py.File(wffile, 'r')
    for k in wf.parameters:
        wf.parameters[k] = np.array(hdf[k])

    accumulators = {'energy': pyqmc.EnergyAccumulator(mol) } 
    #This will get 
    configs = pyqmc.initial_guess(mol, 1000)
    hdf = h5py.File(hdffile,'a')

    pyqmc.vmc(wf, configs, accumulators=accumulators, hdf_file = hdf)


In [5]:
def optimize(chkfile, wffile):
    #recover the mean-field objects
    mol = lib.chkfile.load_mol(chkfile)
    mf = scf.RHF(mol)
    mf.__dict__.update(scf.chkfile.load(chkfile, 'scf'))
    wf, to_opt, freeze = qmcwf(mol, mf)
    accumulator = pyqmc.gradient_generator(mol, wf,to_opt, freeze)
    coords = pyqmc.initial_guess(mol, 1000)
    wf, datagrad, dataline = pyqmc.line_minimization(wf,coords, accumulator)
    hdf = h5py.File(wffile, 'w')
    for k, it in wf.parameters.items():
        hdf.create_dataset(k,data=it)


A simple analysis script to check on the progress of your calculation.

In [6]:
def analyze(hdffile):
    hdf = h5py.File(hdffile, 'r')
    energy = np.array(hdf['energytotal'])
    for nb in [50, 20, 10]:
        reblock = pyqmc.avg_reblock(energy, nb)

        print(nb, np.mean(reblock), np.std(reblock)/np.sqrt(len(reblock)) )


Here we run the mean-field calculation. It's a good idea to run analyze() to make sure that the solution is similar to what you would expect. 

In [9]:
chkfile = 'he.chk'
hdffile = 'he.hdf5'
wffile = "savewf.hdf5"

#This is so we can start over each time we run!
#Don't do this if you want to save your data.
for filenm in [chkfile, hdffile, wffile]:
    if os.path.isfile(filenm):
        os.remove(filenm)
        
mol, mf = mean_field(chkfile)
mf.analyze()

converged SCF energy = -2.86193223020366
**** MO energy ****
MO #1   energy= -0.917801424375274 occ= 2
MO #2   energy= 0.727613019225335  occ= 0
MO #3   energy= 0.727613019225335  occ= 0
MO #4   energy= 0.727613019225338  occ= 0
MO #5   energy= 1.58197375047964   occ= 0
 ** Mulliken atomic charges  **
charge of  0He =      0.00000
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000,  0.00000


((array([1.99999047e+00, 9.53120630e-06, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00]), array([2.22044605e-16])), array([0., 0., 0.]))

We can now run VMC on our Slater-Jastrow wave function. Each time we run it, the error bars will decrease.


In [10]:
optimize(chkfile, wffile)

starting warmup
warmup finished, nsteps 100
descent en -2.871652666545279 0.001473240199867319
descent |grad| 0.15900448767030845
descent step -0.04           -2.873639136    weight stddev 0.01705159428  
descent step 0.02            -2.882616509    weight stddev 0.008436422675 
descent step 0.08            -2.889298999    weight stddev 0.03343174811  
descent step 0.14            -2.893848189    weight stddev 0.05802834375  
descent step 0.2             -2.896416354    weight stddev 0.08231380064  
polynomial fit [ 0.29666101 -0.14210929 -2.87982968]
estimated minimum 0.23951460100260386
estimated minimum adjusted 0.2
descent en -2.892701618606988 0.0008221760949654424
descent |grad| 0.04211285983984151
descent step -0.04           -2.898608638    weight stddev 0.004067577658 
descent step 0.02            -2.900580755    weight stddev 0.002064671723 
descent step 0.08            -2.902389148    weight stddev 0.008384743765 
descent step 0.14            -2.90403662     weight stddev 0.

In [11]:
runvmc(chkfile, wffile, hdffile)
analyze(hdffile)

KeysView(<HDF5 file "he.hdf5" (mode r+)>)
50 -2.8963852 0.0021245231932134627
20 -2.8963847 0.002851327731596174
10 -2.8963847 0.002968451980043285


In [13]:
runvmc(chkfile, wffile, hdffile)
analyze(hdffile)

50 -2.8975494 0.001413637567736277
20 -2.8975496 0.0015544964109144809
10 -2.8975494 0.0015687159187647074


In [14]:
runvmc(chkfile, wffile, hdffile)
analyze(hdffile)

50 -2.8978744 0.0010494981479341658
20 -2.8978746 0.0011378706148400608
10 -2.8978744 0.0011372109743202946
